In [4]:
from dash import Dash, dcc, html, Input, Output, State, ctx
from jupyter_dash import JupyterDash
import os
from IPython.display import clear_output

# ✅ Dash config with assets path
app = JupyterDash(
    __name__,
    assets_folder=os.path.abspath("../assets"),
    assets_url_path="/assets"
)
app.title = "🐞 Insect Carousel"

# 📦 Liste des espèces
INSECT_FOLDER = os.path.abspath("../assets/insects")
INSECT_SPECIES = sorted([
    d for d in os.listdir(INSECT_FOLDER)
    if os.path.isdir(os.path.join(INSECT_FOLDER, d))
])

# 📦 Dictionnaire {species: [image list]}
insect_images = {}
for species in INSECT_SPECIES:
    species_path = os.path.join(INSECT_FOLDER, species)
    images = []
    for root, _, files in os.walk(species_path):
        for f in files:
            if f.lower().endswith((".jpg", ".jpeg", ".png")):
                # Get relative path for browser
                rel_path = os.path.relpath(os.path.join(root, f), INSECT_FOLDER)
                web_path = f"/assets/insects/{rel_path.replace(os.sep, '/')}"
                images.append(web_path)
    insect_images[species] = sorted(images)

# 🖼️ Layout
app.layout = html.Div([
    html.H2("🐞 Insect Carousel", style={"textAlign": "center", "marginBottom": "25px"}),

    html.Div([
        html.Label("Select insect species:", style={
            "fontWeight": "normal", "fontSize": "15px", "marginBottom": "8px"
        }),
        dcc.Dropdown(
            id='species-dropdown',
            options=[{"label": s.replace("_", " ").capitalize(), "value": s} for s in INSECT_SPECIES],
            value=INSECT_SPECIES[0] if INSECT_SPECIES else None,
            clearable=False,
            style={"width": "300px", "margin": "0 auto"}
        )
    ], style={"textAlign": "center", "marginBottom": "20px"}),

    html.Div(id='carousel-container', style={"textAlign": "center"})
])


# 🔁 Carousel update
@app.callback(
    Output('carousel-container', 'children'),
    Input('species-dropdown', 'value')
)
def update_carousel(species):
    images = insect_images.get(species, [])
    if not images:
        return html.P("No images available.", style={"textAlign": "center"})

    filename = os.path.basename(images[0])
    return html.Div([
        html.Div([
            html.Button("⬅️", id="prev-button", n_clicks=0,
                        style={"fontSize": "24px", "padding": "10px 15px", "marginRight": "20px"}),
            html.Img(id="carousel-image", src=images[0],
                     style={"height": "450px", "display": "inline-block", "verticalAlign": "middle"}),
            html.Button("➡️", id="next-button", n_clicks=0,
                        style={"fontSize": "24px", "padding": "10px 15px", "marginLeft": "20px"}),
        ], style={"textAlign": "center", "marginBottom": "20px"}),

        dcc.Store(id='image-index', data=0),
        dcc.Store(id='current-images', data=images),
        html.Div(f"1/{len(images)} - {filename}", id='filename-display',
                 style={"textAlign": "center", "fontSize": "14px", "fontStyle": "italic"})
    ])

# 🔁 Navigation buttons
@app.callback(
    Output("carousel-image", "src"),
    Output("image-index", "data"),
    Output("filename-display", "children"),
    Input("prev-button", "n_clicks"),
    Input("next-button", "n_clicks"),
    State("current-images", "data"),
    State("image-index", "data"),
    prevent_initial_call=True
)
def navigate_images(prev_clicks, next_clicks, images, index):
    if not images:
        return "", 0, "No images"

    triggered = ctx.triggered_id
    if triggered == "next-button":
        index = (index + 1) % len(images)
    elif triggered == "prev-button":
        index = (index - 1) % len(images)

    filename = os.path.basename(images[index])
    return images[index], index, f"{index + 1}/{len(images)} - {filename}"

# 🚀 Launch
clear_output(wait=True)
app.run(port=8454)
